<a href="https://colab.research.google.com/github/esa-prakasa/pandas/blob/master/class12sp_groupC_vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load dan mount Google Drive
from google.colab import drive

# Pilihan user authorization dengan menggunakan account Google
drive.mount('/content/drive')

# Pilihan user authorization dengan menggunakan account Google
!ls -l "/content/drive/My Drive/data12_C/train/"



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
total 48
drwx------ 2 root root 4096 Jul 16 10:11 10091
drwx------ 2 root root 4096 Jul 16 10:11 12672
drwx------ 2 root root 4096 Jul 16 10:11 13381
drwx------ 2 root root 4096 Jul 16 10:11 18212
drwx------ 2 root root 4096 Jul 16 10:11 21684
drwx------ 2 root root 4096 Jul 16 10:11 33497
drwx------ 2 root root 4096 Jul 16 10:11 33963
drwx------ 2 root root 4096 Jul 16 10:11 34013
drwx------ 2 root root 4096 Jul 16 10:11 6171631
drwx------ 2 root root 4096 Ju

In [ ]:
import keras

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import os


img_width, img_height = 100, 100
train_data_dir = "/content/drive/My Drive/data12_C/train/"
validation_data_dir = "/content/drive/My Drive/data12_C/valid/"
nb_train_samples = 30 #4125
nb_validation_samples = 30 #466 
batch_size = 16
epochs = 10

#keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

#model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
model = applications.inception_v3.InceptionV3(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
#for layer in model.layers[:-5]:
#    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(12, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

    
model_final.summary()

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.01), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
#horizontal_flip = True,
#fill_mode = "nearest",
#zoom_range = 0.3,
#width_shift_range = 0.3,
#height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
#horizontal_flip = True,
#fill_mode = "nearest",
#zoom_range = 0.3,
#width_shift_range = 0.3,
#height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")


# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
  train_generator,
  samples_per_epoch = nb_train_samples,
  epochs = epochs,
  validation_data = validation_generator,
  nb_val_samples = nb_validation_samples,
  callbacks = [checkpoint, early])


Using TensorFlow backend.
W0725 09:49:26.906990 139859198986112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 09:49:26.943066 139859198986112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 09:49:26.951494 139859198986112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 09:49:26.993226 139859198986112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0725 

87916544/87910968 [==============================] - 1s 0us/step


W0725 09:49:41.518378 139859198986112 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
W0725 09:49:41.764353 139859198986112 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 49, 49, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 49, 49, 32)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 49, 49, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=10, validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=1, validation_steps=30)`
W0725 09:49:43.247747 139859198986112 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in 

Found 426 images belonging to 12 classes.
Epoch 1/10
1/1 [==============================] - 303s 303s/step - loss: 2.6606 - acc: 0.0625 - val_loss: 2.6439 - val_acc: 0.0688

Epoch 00001: val_acc improved from -inf to 0.06878, saving model to vgg16_1.h5
Epoch 2/10
1/1 [==============================] - 77s 77s/step - loss: 2.6445 - acc: 0.1250 - val_loss: 2.6231 - val_acc: 0.0939

Epoch 00002: val_acc improved from 0.06878 to 0.09389, saving model to vgg16_1.h5
Epoch 3/10
1/1 [==============================] - 109s 109s/step - loss: 2.4702 - acc: 0.0625 - val_loss: 2.5954 - val_acc: 0.0906

Epoch 00003: val_acc did not improve from 0.09389
Epoch 4/10
1/1 [==============================] - 111s 111s/step - loss: 2.6864 - acc: 0.0625 - val_loss: 2.6073 - val_acc: 0.0906

Epoch 00004: val_acc did not improve from 0.09389
Epoch 5/10
1/1 [==============================] - 109s 109s/step - loss: 2.5979 - acc: 0.1875 - val_loss: 2.6135 - val_acc: 0.0786

Epoch 00005: val_acc did not improve fr